# Articles
https://keras.io/guides/transfer_learning/
https://keras.io/guides/training_with_built_in_methods/
https://www.youtube.com/watch?v=4umFSRPx-94&ab_channel=DigitalSreeni
https://www.tensorflow.org/guide/saved_model
https://github.com/UCSD-E4E/PyHa/blob/Microfaune_Retraining/Microfaune_Retraining-Copy1.ipynb

In [1]:
%load_ext autoreload
%autoreload 2

In [23]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, Flatten, ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import numpy as np

In [24]:
train = True

## Load in BirdNET-Analyzer Model

In [4]:
# path to model folder, should have assets/variables/frozen pb graphs
path_to_saved_model = "./checkpoints/V2.1/BirdNET_GLOBAL_2K_V2.1_Model"

In [5]:
model = keras.models.load_model(path_to_saved_model)

In [26]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=['accuracy'])

In [27]:
x = model.layers[-2].output
o = tf.keras.layers.Dense(len(species_of_interests), activation='sigmoid', name='its_new_lmao')(x)

model3 = Model(inputs=model.input, outputs=o)


In [28]:
model_extended = model3
model_extended

In [29]:
model_extended.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
INPUT (InputLayer)              [(None, 144000)]     0                                            
__________________________________________________________________________________________________
ADVANCED_SPEC1 (LinearSpecLayer (None, 128, 513, 1)  1           INPUT[0][0]                      
__________________________________________________________________________________________________
BNORM_SPEC_NOQUANT (BatchNormal (None, 128, 513, 1)  4           ADVANCED_SPEC1[0][0]             
__________________________________________________________________________________________________
CONV_0 (Conv2D)                 (None, 64, 257, 30)  960         BNORM_SPEC_NOQUANT[0][0]         
_______________________________________________________________________________________

In [34]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)


Num GPUs Available:  0


## Preprocess and load data
According to `config.py` and `checkpoints/README.md` for V2.1:  
- Model training input size: 144000 = 48000 x 3 = sample rate x num chunks  
- Model training output size: 2434 = 2424 bird classes + 10 non-event classes
- Visualize using [Netron](https://netron.app/)

In [8]:
import librosa

def splitSignal(sig, rate, overlap, seconds=3.0, minlen=1.5):

    # Split signal with overlap
    sig_splits = []
    for i in range(0, len(sig), int((seconds - overlap) * rate)):
        split = sig[i:i + int(seconds * rate)]

        # End of signal?
        if len(split) < int(minlen * rate):
            break
        
        # Signal chunk too short? Fill with zeros.
        if len(split) < int(rate * seconds):
            temp = np.zeros((int(rate * seconds)))
            temp[:len(split)] = split
            split = temp
        
        sig_splits.append(split)

    return sig_splits

def readAudioData(path, overlap, sample_rate=48000):

    #print('READING AUDIO DATA...', end=' ', flush=True)

    # Open file with librosa (uses ffmpeg or libav)
    try:
        sig, rate = librosa.load(path, sr=sample_rate, mono=True, res_type='kaiser_fast')
        clip_length = librosa.get_duration(y=sig, sr=rate)
    except:
        return 0
    # Split audio into 3-second chunks
    chunks = splitSignal(sig, rate, overlap)

    print('DONE! READ', str(len(chunks)), 'CHUNKS.')

    return chunks, clip_length

import pandas as pd
import numpy as np


#TODO Add mutliple species support for ROC curves 
def annotation_chunker_no_duplicates(kaleidoscope_df, chunk_length, include_no_bird=False, bird=None):
    """
    Function that converts a Kaleidoscope-formatted Dataframe containing 
    annotations to uniform chunks of chunk_length. If there
    are mutliple bird species in the same clip, this function creates chunks
    for the more confident bird species.

    Note: if all or part of an annotation covers the last < chunk_length
    seconds of a clip it will be ignored. If two annotations overlap in 
    the same 3 second chunk, both are represented in that chunk
    Args:
        kaleidoscope_df (Dataframe)
            - Dataframe of annotations in kaleidoscope format

        chunk_length (int)
            - duration to set all annotation chunks
    Returns:
        Dataframe of labels with chunk_length duration 
        (elements in "OFFSET" are divisible by chunk_length).
    """

    #Init list of clips to cycle through and output dataframe
    #kaleidoscope_df["FILEPATH"] =  kaleidoscope_df["FOLDER"] + kaleidoscope_df["IN FILE"] 
    kaleidoscope_df['FILEPATH'] = kaleidoscope_df.loc[:,['FOLDER','IN FILE']].sum(axis=1)
    clips = kaleidoscope_df["FILEPATH"].unique()
    df_columns = {'FOLDER': 'str', 'IN FILE' :'str', 'CLIP LENGTH' : 'float64', 'CHANNEL' : 'int64', 'OFFSET' : 'float64',
                'DURATION' : 'float64', 'SAMPLE RATE' : 'int64','MANUAL ID' : 'str'}
    output_df = pd.DataFrame({c: pd.Series(dtype=t) for c, t in df_columns.items()})
    
    # going through each clip
    for clip in clips:
        clip_df = kaleidoscope_df[kaleidoscope_df["FILEPATH"] == clip]
        path = clip_df["FOLDER"].unique()[0]
        file = clip_df["IN FILE"].unique()[0]
        birds = clip_df["MANUAL ID"].unique()
        sr = clip_df["SAMPLE RATE"].unique()[0]
        clip_len = clip_df["CLIP LENGTH"].unique()[0]

        # quick data sanitization to remove very short clips
        # do not consider any chunk that is less than chunk_length
        if clip_len < chunk_length:
            continue
        potential_annotation_count = int(clip_len)//int(chunk_length)

        # going through each species that was ID'ed in the clip
        arr_len = int(clip_len*1000)
        species_df = clip_df#[clip_df["MANUAL ID"] == bird]
        human_arr = np.zeros((arr_len))
        # looping through each annotation
        #print("========================================")
        for annotation in species_df.index:
            #print(species_df["OFFSET"][annotation])
            minval = int(round(species_df["OFFSET"][annotation] * 1000, 0))
            # Determining the end of a human label
            maxval = int(
                round(
                    (species_df["OFFSET"][annotation] +
                        species_df["DURATION"][annotation]) *
                    1000,
                    0))
            # Placing the label relative to the clip
            human_arr[minval:maxval] = 1
        # performing the chunk isolation technique on the human array

        for index in range(potential_annotation_count):
            #print("=======================")
            #print("-----------------------------------------")
            #print(index)
            chunk_start = index * (chunk_length*1000)
            chunk_end = min((index+1)*chunk_length*1000,arr_len)
            chunk = human_arr[int(chunk_start):int(chunk_end)]
            if max(chunk) >= 0.5:
                #Get row data
                row = pd.DataFrame(index = [0])
                annotation_start = chunk_start / 1000

                #Handle birdnet output edge case
                #print("-------------------------------------------")
                #print(sum(clip_df["DURATION"] == 3))
                #print(sum(clip_df["DURATION"] == 3)/clip_df.shape[0])
                #print("-------------------------------------------")
                if(sum(clip_df["DURATION"] == 3)/clip_df.shape[0] == 1):
                    #print("Processing here duration")
                    overlap = (clip_df["OFFSET"]+0.5 >= (annotation_start)) & (clip_df["OFFSET"]-0.5 <= (annotation_start))
                    annotation_df = clip_df[overlap]
                    #print(annotation_start, np.array(clip_df["OFFSET"]), overlap)
                    #print(annotation_df)
                else:
                    #print("Processing here")
                    overlap = is_overlap(clip_df["OFFSET"], clip_df["OFFSET"] + clip_df["DURATION"], annotation_start, annotation_start + chunk_length)
                    #print(overlap)
                    annotation_df = clip_df[overlap]
                    #print(annotation_df)
                
                #updating the dictionary
                if ('CONFIDENCE' in clip_df.columns):
                    annotation_df = annotation_df.sort_values(by="CONFIDENCE", ascending=False)
                    row["CONFIDENCE"] = annotation_df.iloc[0]["CONFIDENCE"]
                else:
                    #The case of manual id, or there is an annotation with no known confidence
                    row["CONFIDENCE"] = 1
                row["FOLDER"] = path
                row["IN FILE"] = file
                row["CLIP LENGTH"] = clip_len
                row["OFFSET"] = annotation_start
                row["DURATION"] = chunk_length
                row["SAMPLE RATE"] = sr
                row["MANUAL ID"] = annotation_df.iloc[0]["MANUAL ID"] 
                row["CHANNEL"] = 0
                output_df = pd.concat([output_df,row], ignore_index=True)
            elif(include_no_bird):
                #print(max(chunk))
                #Get row data
                row = pd.DataFrame(index = [0])
                annotation_start = chunk_start / 1000

                #updating the dictionary
                row["CONFIDENCE"] = 0
                row["FOLDER"] = path
                row["IN FILE"] = file
                row["CLIP LENGTH"] = clip_len
                row["OFFSET"] = annotation_start
                row["DURATION"] = chunk_length
                row["SAMPLE RATE"] = sr
                row["MANUAL ID"] = "no bird"
                row["CHANNEL"] = 0
                output_df = pd.concat([output_df,row], ignore_index=True)
    
    return output_df


def is_overlap(offset_df, end_df, chunk_start, chunk_end):
    is_both_before = (chunk_end < offset_df) & (chunk_start < offset_df)
    is_both_after = (end_df < chunk_end) & (end_df < chunk_start)
    return (~is_both_before) & (~is_both_after)
    
    
    interval = pd.Interval(left=offset_df, right=end_df)
    print(interval)


def split_save_files(dataframe_of_data):
    chunked_df = annotation_chunker_no_duplicates(dataframe_of_data, 3, include_no_bird=True)
    
    
    chunked_df.apply()
    



In [10]:
#chunks,clip_length  = readAudioData("./Data/code-testing/XC628854 - Yellow-olive Flatbill - Tolmomyias sulphurescens.mp3", 0, sample_rate=48000)

In [13]:
import pandas as pd
data = pd.read_csv("C:/Users/seanh/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing/automated_cosmos_tweety_to_birdnet.csv")
chunked_df = annotation_chunker_no_duplicates(data, 3, include_no_bird=True)
chunked_df

,FOLDER,IN FILE,CLIP LENGTH,CHANNEL,OFFSET,DURATION,SAMPLE RATE,MANUAL ID,CONFIDENCE
0,.\..\Cosmos_data\,XC100027 - Southern Nightingale-Wren - Microce...,81.528167,0,0.0,3.0,48000,Microcerculus marginatus,0.155038
1,.\..\Cosmos_data\,XC100027 - Southern Nightingale-Wren - Microce...,81.528167,0,3.0,3.0,48000,Centropus celebensis,0.020985
2,.\..\Cosmos_data\,XC100027 - Southern Nightingale-Wren - Microce...,81.528167,0,6.0,3.0,48000,Meliphaga notata,0.027436
3,.\..\Cosmos_data\,XC100027 - Southern Nightingale-Wren - Microce...,81.528167,0,9.0,3.0,48000,Microcerculus marginatus,0.331647
4,.\..\Cosmos_data\,XC100027 - Southern Nightingale-Wren - Microce...,81.528167,0,12.0,3.0,48000,Sciaphylax hemimelaena,0.017806
...,...,...,...,...,...,...,...,...,...
39712,.\..\Cosmos_data\,XC9881 - Blue-grey Tanager - Thraupis episcopu...,14.700021,0,0.0,3.0,48000,Thraupis episcopus,0.146423
39713,.\..\Cosmos_data\,XC9881 - Blue-grey Tanager - Thraupis episcopu...,14.700021,0,3.0,3.0,48000,Pyrrhocorax pyrrhocorax,0.042745
39714,.\..\Cosmos_data\,XC9881 - Blue-grey Tanager - Thraupis episcopu...,14.700021,0,6.0,3.0,48000,Thraupis sayaca,0.108350
39715,.\..\Cosmos_data\,XC9881 - Blue-grey Tanager - Thraupis episcopu...,14.700021,0,9.0,3.0,48000,no bird,0.000000


In [18]:
to_file = pd.read_csv("C:/Users/seanh/Desktop/e4e/PyHa/automated_cosmos_tweety_to_file.csv")
classes_of_interest = np.unique(to_file["MANUAL ID"])
chunked_df = chunked_df[(chunked_df["MANUAL ID"].isin(classes_of_interest))]#["MANUAL_ID"] = "bird"
len(np.unique(chunked_df["MANUAL ID"]))

10

In [19]:
chunked_df["FOLDER"] = "./Data/code-testing/"

In [55]:
species_of_interests = list(np.unique(chunked_df["MANUAL ID"]))
chunked_df["y"] = chunked_df["MANUAL ID"].apply(lambda x: species_of_interests.index(x))
chunked_df

X = []
Y = []
UID = []

#assume df is chunked
for file in np.unique(chunked_df["IN FILE"]):
    file_df = chunked_df[chunked_df["IN FILE"] == file]
    #print((file_df["FOLDER"] + file_df["IN FILE"]).iloc[0])
    try:
        chunks, clip_length = readAudioData((file_df["FOLDER"] + file_df["IN FILE"]).iloc[0], 0, sample_rate=48000)
    except:
        continue
    offset = 0
    for c in range(len(chunks)):
        offset = c * 3
        try:
            tmp = file_df[file_df["OFFSET"] == offset]

            if (tmp.empty):
                continue
            chunk_df = tmp.sample(1)
            if (chunk_df.empty):
                continue
                #Wouldn't this miss no bird??????
            # Add to batch
            X.append(np.array(chunks[c]))
            Y.append(chunk_df["y"].iloc[0])
            UID.append(chunk_df.index[0])
        except Exception as e:
            print(e)
            print(file_df)
            continue

        
X, Y, UID

DONE! READ 14 CHUNKS.
                     FOLDER  \
35093  ./Data/code-testing/   

                                                 IN FILE  CLIP LENGTH  \
35093  XC691283 - White-throated Toucan - Ramphastos ...    41.743688   

       CHANNEL  OFFSET  DURATION  SAMPLE RATE           MANUAL ID  CONFIDENCE  \
35093        0     0.0       3.0        48000  Ramphastos tucanus    0.925904   

       y  
35093  3  
                     FOLDER  \
35093  ./Data/code-testing/   

                                                 IN FILE  CLIP LENGTH  \
35093  XC691283 - White-throated Toucan - Ramphastos ...    41.743688   

       CHANNEL  OFFSET  DURATION  SAMPLE RATE           MANUAL ID  CONFIDENCE  \
35093        0     0.0       3.0        48000  Ramphastos tucanus    0.925904   

       y  
35093  3  
                     FOLDER  \
35094  ./Data/code-testing/   

                                                 IN FILE  CLIP LENGTH  \
35094  XC691283 - White-throated Toucan - Ramphastos

DONE! READ 7 CHUNKS.
                     FOLDER  \
35106  ./Data/code-testing/   

                                                 IN FILE  CLIP LENGTH  \
35106  XC691284 - White-throated Toucan - Ramphastos ...    21.106958   

       CHANNEL  OFFSET  DURATION  SAMPLE RATE           MANUAL ID  CONFIDENCE  \
35106        0     0.0       3.0        48000  Ramphastos tucanus    0.804653   

       y  
35106  3  
                     FOLDER  \
35106  ./Data/code-testing/   

                                                 IN FILE  CLIP LENGTH  \
35106  XC691284 - White-throated Toucan - Ramphastos ...    21.106958   

       CHANNEL  OFFSET  DURATION  SAMPLE RATE           MANUAL ID  CONFIDENCE  \
35106        0     0.0       3.0        48000  Ramphastos tucanus    0.804653   

       y  
35106  3  
                     FOLDER  \
35107  ./Data/code-testing/   

                                                 IN FILE  CLIP LENGTH  \
35107  XC691284 - White-throated Toucan - Ramphastos 

DONE! READ 9 CHUNKS.
                     FOLDER  \
36742  ./Data/code-testing/   

                                                 IN FILE  CLIP LENGTH  \
36742  XC710030 - White-throated Toucan - Ramphastos ...    25.887979   

       CHANNEL  OFFSET  DURATION  SAMPLE RATE           MANUAL ID  CONFIDENCE  \
36742        0     0.0       3.0        48000  Ramphastos tucanus    0.301793   

       y  
36742  3  
                     FOLDER  \
36742  ./Data/code-testing/   

                                                 IN FILE  CLIP LENGTH  \
36742  XC710030 - White-throated Toucan - Ramphastos ...    25.887979   

       CHANNEL  OFFSET  DURATION  SAMPLE RATE           MANUAL ID  CONFIDENCE  \
36742        0     0.0       3.0        48000  Ramphastos tucanus    0.301793   

       y  
36742  3  
                     FOLDER  \
36743  ./Data/code-testing/   

                                                 IN FILE  CLIP LENGTH  \
36743  XC710030 - White-throated Toucan - Ramphastos 

TypeError: array2string() missing 1 required keyword-only argument: 'legacy'

In [64]:
print(X)

False


TypeError: dragon4_scientific() takes at most 7 arguments (8 given)

In [44]:
dataset = {"X":X, "Y": Y, "uids": UID}
inds = [i for i, x in enumerate(dataset["X"])]
X = np.array([dataset["X"][i].transpose() for i in inds])
Y = np.array([int(dataset["Y"][i]) for i in inds])
uids = [dataset["uids"][i] for i in inds]
X, Y, uids

TypeError: array2string() missing 1 required keyword-only argument: 'legacy'

In [45]:
from sklearn.model_selection import StratifiedShuffleSplit
def split_dataset(X, Y, test_size=0.2, random_state=0):
    split_generator = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    ind_train, ind_test = next(split_generator.split(X, Y))
    X_train, X_test = X[ind_train], X[ind_test]
    Y_train, Y_test = Y[ind_train], Y[ind_test]
    return ind_train, ind_test

ind_train_val, ind_test = split_dataset(X, Y)


ValueError: Only callable can be used as callback

In [ ]:
ind_train_val, ind_test

In [ ]:
#TODO ACTUALLY CREATE A VALIDATION TEST SET
from collections import Counter
X_train, Y_train = X[ind_train_val, np.newaxis], Y[ind_train_val]
X_test,  Y_test =  X[ind_test, np.newaxis], Y[ind_test]          
                    

print("Training set: ", Counter(Y_train))
print("Test set: ", Counter(Y_test))
#print("Validation set: ", Counter(Y_val))
#very imbalanced dataset

In [ ]:
species_of_interests

## Retrain Model

In [ ]:
from datetime import datetime
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, Y, batch_size=32, output_size=10):
        self.X = X
        self.Y = Y
        self.n = len(Y)
        self.batch_size = batch_size
        self.output_size = output_size
        self.shuffle()
        
        
    def __len__(self):
        print("RUNNINGS", int(np.floor(self.n)/self.batch_size))
        return int(np.floor(self.n)/self.batch_size)
    
    def __getitem__(self, index):
        
        batch_inds = self.inds[self.batch_size*index:self.batch_size*(index+1)]
        #print(batch_inds)
        self.counter += self.batch_size
        if self.counter >= self.n:
            self.shuffle()
            
        #TODO: FIX SO BATCH IS MORE THAN 1
        raw_labels = np.array([self.Y[batch_inds][0]])
        formatted_labels = np.zeros(self.output_size)
        formatted_labels[self.Y[batch_inds][0]] = 1
        fprmatted_labels = np.array([formatted_labels])
        #print(formatted_labels.shape)
        #print(np.array([formatted_labels]).shape)
        #print(self.X[batch_inds, ...][0].shape)
        
        #print(tf.convert_to_tensor(self.X[batch_inds, ...][0]), tf.convert_to_tensor(np.array([formatted_labels])))
        return tf.convert_to_tensor(self.X[batch_inds, ...][0]), tf.convert_to_tensor(np.array([formatted_labels]))
    
    def shuffle(self):
        self.inds = np.random.permutation(self.n)
        self.counter = 0
# if train:

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, Flatten, ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import numpy as np

# microfaune training script

if train:
    optimizer = keras.optimizers.Adam(lr=0.001)
    model_extended.compile(optimizer=optimizer,
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy', keras.metrics.FalseNegatives()])

    alpha = 0.5
    batch_size = 1
    date_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    data_generator = DataGenerator(X_train, Y_train, batch_size)
    
    micro_callbacks = [
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                  patience=5, min_lr=1e-5),
        #keras.callbacks.ModelCheckpoint('microfaune-' + date_str +'-{epoch:02d}.h5',
        #                          save_weights_only=False)
    ]
    
    #validation_data=(X_test, Y_test),
    
    history = model_extended.fit(data_generator, epochs=10,batch_size=1,
                                  #validation_data=(X_test, Y_test), class_weight={0: alpha, 1: 1-alpha},callbacks=micro_callbacks,
                                   verbose=2)
    
    #model.save(f"model-{date_str}")
    #model.save_weights(f"model_weights-{date_str}.h5")

# Testing Model

In [ ]:
from datetime import datetime
class ValidDataset(keras.utils.Sequence):
    def __init__(self, X, Y, batch_size=32, output_size=10):
        self.X = X
        self.Y = Y
        #self.UIDs = UIDs
        self.n = len(Y)
        self.batch_size = batch_size
        self.output_size = output_size
        self.shuffle()
        
        
    def __len__(self):
        print("RUNNINGS", int(np.floor(self.n)/self.batch_size))
        return int(np.floor(self.n)/self.batch_size)
    
    def len_of_labels(self):
        return self.output_size
    
    def __getitem__(self, index):
        
        batch_inds = self.inds[self.batch_size*index:self.batch_size*(index+1)]
        #print(batch_inds)
        self.counter += self.batch_size
        if self.counter >= self.n:
            self.shuffle()
            
        #TODO: FIX SO BATCH IS MORE THAN 1
        raw_labels = np.array([self.Y[batch_inds][0]])
        formatted_labels = np.zeros(self.output_size)
        formatted_labels[self.Y[batch_inds][0]] = 1
        fprmatted_labels = np.array([formatted_labels])
        #print(formatted_labels.shape)
        #print(np.array([formatted_labels]).shape)
        #print(self.X[batch_inds, ...][0].shape)
        
        #print(tf.convert_to_tensor(self.X[batch_inds, ...][0]), tf.convert_to_tensor(np.array([formatted_labels])))
        return tf.convert_to_tensor(self.X[batch_inds, ...][0]), tf.convert_to_tensor(np.array([formatted_labels]))
    
    def shuffle(self):
        self.inds = np.random.permutation(self.n)
        self.counter = 0
# if train:

In [ ]:
validate_dataset = ValidDataset(X_train, Y_train, batch_size)

In [ ]:
predictions = model_extended.predict(
    validate_dataset.__getitem__(0)[0],
    batch_size=None,
    verbose='1',
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False
)

In [ ]:
len(predictions)

In [ ]:
validate_dataset.__getitem__(0)[1]

In [ ]:
 label_df = pd.DataFrame(columns=range(validate_dataset.len_of_labels()))

In [ ]:
#Label_df_one_hot_encoding
label_df = pd.DataFrame(columns=range(validate_dataset.len_of_labels()))
scores_df = pd.DataFrame(columns=range(validate_dataset.len_of_labels()))
preds_df = pd.DataFrame(columns=range(validate_dataset.len_of_labels()))
for i in range(len(validate_dataset)):
    #print(i)
    predictions = model_extended.predict(
        validate_dataset.__getitem__(i)[0],
        batch_size=None,
        verbose='1',
        steps=None,
        callbacks=None,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=False
    )
    
    label = predictions.argmax()
    label_arr = np.zeros(validate_dataset.len_of_labels())
    label_arr[label] = 1
    
    preds_df = preds_df.append(pd.DataFrame(np.array([label_arr])))
    scores_df = scores_df.append(pd.DataFrame(predictions))
    label_df = label_df.append(pd.DataFrame(validate_dataset.__getitem__(i)[1].numpy()))
    
preds_df.columns = species_of_interests
scores_df.columns = species_of_interests
label_df.columns = species_of_interests

In [ ]:
preds_df

In [ ]:
scores_df

In [ ]:
label_df

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

In [ ]:
print(classification_report(label_df, preds_df))

In [ ]:
import matplotlib.pyplot as plt
for species in species_of_interests:
    fpr, tpr, thresh = roc_curve(label_df[species],  scores_df[species])
    auc = roc_auc_score(label_df[species],  preds_df[species])
    plt.plot(fpr,tpr,label="AUC " + species + " "+str(auc))

plt.title('Classwise ROC Curves')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
plt.show()